In [25]:
#import modules
import os, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import colicycle.time_mat_operations as tmo
from colicycle.MoMAobj import Momaobj
import colicycle.tools_GW as tgw

In [26]:
folder_prefix = '../PreProcessed/'

In [27]:
experiment_info = [
    {'data_folder':folder_prefix+'20180709_GW296_glucose8aa37_1_MMStack/',
     'time_scale': 3, 'switches_h':[], 'switches':[]},
    {'data_folder':folder_prefix+'20180711_GW296_glucose37_1_MMStack/',
     'time_scale': 3, 'switches_h':[],'switches':[]},
    {'data_folder':folder_prefix+'20180706_GW296_glycerol37_1_MMStack/',
     'time_scale': 3, 'switches_h':[],'switches':[]},
    {'data_folder':folder_prefix+'20170327_GW339_temp/',
     'time_scale': 3, 'switches_h':[2,17.10,40.08],'switches':[]}
]

tosave_folder = folder_prefix


In [28]:
#the 20170327_GW339_temp experiment contains additional data where temperature was changed.
#We calculate the switch times in frame units to keep only the firs part
for x in experiment_info:
    if len(x['switches_h'])>0:
        x['switches'] = np.round((np.floor(x['switches_h'])*60+100*(x['switches_h']-np.floor(x['switches_h'])))/x['time_scale']).astype(int)


In [30]:
for data_ind in range(len(experiment_info)):
    
    expinfo = experiment_info[data_ind]
    dataset = experiment_info[data_ind]['data_folder']
    
    current_name = dataset.split('/')[np.argwhere([re.findall('^[0-9]{8,9}', x) for x in dataset.split('/')])[0][0]]

    exports = [f for f in os.listdir(tosave_folder+current_name+'/step2') if re.search('(?<=time_mat_pos).*(\d+)_GL(\d+).pkl', f)]
    
    #go through each growht lane and calcualte initiation parameters
    for x in exports:
        time_mat_pd = pd.read_pickle(tosave_folder+current_name+'/step2/'+x)

        datapos = re.findall('(?<=time_mat_pos).*(\d+)_GL(\d+).pkl',x)[0]

        #recover length at initiation
        time_mat_pd['Li'] = np.nan
        time_mat_pd['Li_fit'] = np.nan

        for ind in time_mat_pd.index:
            if not np.isnan(time_mat_pd.at[ind,'Ti']):
                if time_mat_pd.at[ind,'Ti']>=0:
                    time_mat_pd.at[ind,'Li'] = time_mat_pd.at[ind,'length'][int(time_mat_pd.at[ind,'Ti'])]

                    time_mat_pd.at[ind,'Li_fit'] = tgw.fun_expgrowht2(time_mat_pd.at[ind,'Ti'],
                                                                      time_mat_pd.at[ind,'Lb_fit'],time_mat_pd.at[ind,'tau_fit'])

                else:
                    mother_index = time_mat_pd.at[ind,'mother_id']
                    real_time = len(time_mat_pd.at[mother_index,'length'])+int(time_mat_pd.at[ind,'Ti'])
                    time_mat_pd.at[ind,'Li'] = time_mat_pd.at[mother_index,'length'][int(time_mat_pd.at[ind,'Ti'])]/2

                    time_mat_pd.at[ind,'Li_fit'] = tgw.fun_expgrowht2(real_time,time_mat_pd.at[mother_index,'Lb_fit'],
                                                                      time_mat_pd.at[mother_index,'tau_fit'])/2


        filename = tosave_folder+current_name+'/step3/initlen_time_mat'+'_pos'+datapos[0]+'_GL'+datapos[1]+'.pkl'
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        time_mat_pd.to_pickle(filename)
        
    #recover all exports
    exports = [f for f in os.listdir(tosave_folder+current_name+'/step3') if re.search('(?<=initlen_time_mat_pos).*(\d+)_GL(\d+).pkl', f)]
    mats = []
    for x in exports:
        mats.append(pd.read_pickle(tosave_folder+current_name+'/step3/'+x)[['length','born','tau_fit','mother_id','Lb','Lb_fit','Ld_fit','Ld','Li','Li_fit','Ti','Td','full_cellcycle','pearson_log']])

    #combine all exports and measure initiation adder
    cumcells = np.cumsum([0]+[len(x) for x in mats])
    for ind, m in enumerate(mats):
        for x in m.index:
            if m.at[x,'mother_id']>0:
                m.at[x,'DLi'] = m.at[x,'Li_fit']-0.5*m.at[m.at[x,'mother_id'],'Li_fit']
                m.at[x,'Li_old'] = 0.5*m.at[m.at[x,'mother_id'],'Li_fit']

        m.mother_id = m.mother_id.apply(lambda x: x+cumcells[ind] if x>0 else x)
    colidata = pd.concat(mats)
    colidata['period'] = 0
    
    colidata.period.astype('category')
    for x in range(len(expinfo['switches'])):
        colidata.loc[colidata.born > expinfo['switches'][x],'period'] = x+1

    colidata[['born','tau_fit','Ti','Td']]=colidata[['born','tau_fit','Ti','Td']].applymap(lambda x: x*expinfo['time_scale'])

    
    #re-index dataframe and keep complete cell cylces only
    colidata.index = np.arange(len(colidata))
    colidata = colidata[colidata.full_cellcycle]
    
    colidata.to_pickle(tosave_folder+current_name+'/colidata.pkl')
    

In [20]:
time_mat_pd = pd.read_pickle(tosave_folder+'20180711_GW296_glucose37_1_MMStack/'+'/step2/'+'manual_time_mat_pos0_GL06.pkl')


In [18]:
time_mat_pd = pd.read_pickle('/Users/gw18g940/GoogleDrive/PostDocBasel/ColiPublication_GDrive/DoubleAdderCode/Data/20180709_GW296_glucose8aa37_1_MMStack/processed/step1_time_mat_pos0_GL01.pkl')
